## CUSTOMIZATION
---

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()
# подключить GPU: Runtime -> Change runtime type -> Hardware accelerator: GPU
# вывouyод: '/device:GPU:0'

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# монтируем гугл диск

In [0]:
# копируем и разархивируем файлы в colab
! cp /content/gdrive/'My Drive'/Mouse/dataset.zip . # тут точка!
! unzip -q dataset
! rm dataset.zip
! ls

## FCNN
---

### Libraries
---

In [0]:
from tensorflow.keras.models import Sequential, Model, save_model, load_model
from tensorflow.keras.layers import (Input, Conv1D, UpSampling1D, MaxPool1D,
                                     BatchNormalization, Dropout, ReLU)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import logcosh, mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.preprocessing import StandardScaler
import numpy as np
import glob
import os

### Global variables
---


In [0]:
N_EXPERIMENT = 1

MODEL_NAME = f'{N_EXPERIMENT}_mouse_model'
WEIGHT_NAME = f'{N_EXPERIMENT}_mouse_weight'
LOG_NAME = f'{N_EXPERIMENT}_training_log.csv'
GDRIVE_PATH = f'./gdrive/My Drive/Mouse/{N_EXPERIMENT}'
CHECKPOINT_PATH = f'{GDRIVE_PATH}/{WEIGHT_NAME}''_{epoch:03d}_loss-{loss:.3f}_valloss-{val_loss:.3f}.h5'
N_FEATURES = 79
BATCH_SIZE = 8

### Load data
---

Option:

In [0]:
DATASET = 'BALABIT'
USERNAME = 'user07'
SESSION = 'session_all'

In [0]:
def load_data(dataset: str,
              username: str,
              session: str,
              mode: str,
              legal: bool = True) -> np.array:
    if legal:
        path = f"./dataset/{dataset}/{mode}_features/{username}/{session}"
        X = pd.read_csv(path, sep=',', header=None).values
    else:
        X = None
        for path in glob.glob(f"./dataset/{dataset}/{mode}_features/user*"):
            if os.path.basename(path) == username:
                continue
            session_path = os.path.join(path, session)
            features = pd.read_csv(session_path, sep=',', header=None).values
            X = np.vstack((X, features)) if X is not None else features
    X = StandardScaler().fit_transform(X)
    return X

Train data:

In [12]:
X = load_data(DATASET, USERNAME, SESSION, 'train')
N_FEATURES = X.shape[1]
print(f"Train data shape: {X.shape}")

Train data shape: (3417, 79)


Validation data:

In [26]:
X_valid = load_data(DATASET, USERNAME, SESSION, 'test')
print(f"Validation data shape: {X_valid.shape}")

Validation data shape: (3457, 79)


### Model
---

In [0]:
from tensorflow.keras.layers import RepeatVector, Flatten

In [0]:
def get_model(n_features): # "FCNN"
    model = Sequential()
    model.add(Input(shape=(n_features, )))
    model.add(RepeatVector(n_features))

    model.add(Conv1D(256, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(256, 3, padding='same', activation=ReLU()))
    model.add(MaxPool1D())

    model.add(BatchNormalization())
    model.add(Conv1D(128, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(128, 3, padding='same', activation=ReLU()))
    model.add(MaxPool1D())

    model.add(BatchNormalization())
    model.add(Conv1D(64, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(64, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(64, 3, padding='same', activation=ReLU()))
    model.add(MaxPool1D())

    model.add(BatchNormalization())
    model.add(Conv1D(32, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(32, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(32, 3, padding='same', activation=ReLU()))
    model.add(MaxPool1D())

    model.add(BatchNormalization())
    model.add(UpSampling1D())
    model.add(Conv1D(32, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(32, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(32, 3, padding='same', activation=ReLU()))

    model.add(BatchNormalization())
    model.add(UpSampling1D())
    model.add(Conv1D(64, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(64, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(64, 3, padding='same', activation=ReLU()))

    model.add(BatchNormalization())
    model.add(UpSampling1D())
    model.add(Conv1D(128, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(128, 3, padding='same', activation=ReLU()))
    
    model.add(BatchNormalization())
    model.add(UpSampling1D())
    model.add(Conv1D(256, 3, padding='same', activation=ReLU()))
    model.add(Conv1D(256, 3, padding='same', activation=ReLU()))

    model.add(Flatten())
    model.add(Dense(units=256, activation=ReLU()))
    model.add(Dense(units=n_features))

    return model

In [0]:
model = get_model(N_FEATURES)
model.summary()

Compile:

In [0]:
model.compile(optimizer=Adam(lr=0.001),
              loss=logcosh,
              metrics=['mse'])

Callbacks:

In [0]:
checkpointer = ModelCheckpoint(filepath=CHECKPOINT_PATH, monitor='val_loss',
                               verbose=1, save_best_only=True,
                               save_weights_only=True, mode='min')
logger = CSVLogger(LOG_NAME)

Fit:

In [0]:
model.fit(X[:, :], X[:, :],
          batch_size=16,
          epochs=100,
          verbose=1,
        #   callbacks=[logger, checkpointer],
          validation_data=(X_valid[:, :], X_valid[:, :]),
          shuffle=False)

### Evaluate
---

In [0]:
y_test, X_test = load_data(DATASET, USERNAME, SESSION, 'train',
                           legal=False, timesteps=timesteps)

In [0]:
y_pred = model.predict(X)